# Sarcina de clasificare a textului

În acest modul, vom începe cu o sarcină simplă de clasificare a textului bazată pe setul de date **[AG_NEWS](http://www.di.unipi.it/~gulli/AG_corpus_of_news_articles.html)**: vom clasifica titlurile știrilor în una dintre cele 4 categorii: Lume, Sport, Afaceri și Știință/Tehnologie.

## Setul de date

Pentru a încărca setul de date, vom folosi API-ul **[TensorFlow Datasets](https://www.tensorflow.org/datasets)**.


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds

# In this tutorial, we will be training a lot of models. In order to use GPU memory cautiously,
# we will set tensorflow option to grow GPU memory allocation when required.
physical_devices = tf.config.list_physical_devices('GPU') 
if len(physical_devices)>0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

dataset = tfds.load('ag_news_subset')

Putem acum să accesăm părțile de antrenament și testare ale setului de date folosind `dataset['train']` și `dataset['test']` respectiv:


In [3]:
ds_train = dataset['train']
ds_test = dataset['test']

print(f"Length of train dataset = {len(ds_train)}")
print(f"Length of test dataset = {len(ds_test)}")

Length of train dataset = 120000
Length of test dataset = 7600


Să imprimăm primele 10 titluri noi din setul nostru de date:


In [4]:
classes = ['World', 'Sports', 'Business', 'Sci/Tech']

for i,x in zip(range(5),ds_train):
    print(f"{x['label']} ({classes[x['label']]}) -> {x['title']} {x['description']}")

3 (Sci/Tech) -> b'AMD Debuts Dual-Core Opteron Processor' b'AMD #39;s new dual-core Opteron chip is designed mainly for corporate computing applications, including databases, Web services, and financial transactions.'
1 (Sports) -> b"Wood's Suspension Upheld (Reuters)" b'Reuters - Major League Baseball\\Monday announced a decision on the appeal filed by Chicago Cubs\\pitcher Kerry Wood regarding a suspension stemming from an\\incident earlier this season.'
2 (Business) -> b'Bush reform may have blue states seeing red' b'President Bush #39;s  quot;revenue-neutral quot; tax reform needs losers to balance its winners, and people claiming the federal deduction for state and local taxes may be in administration planners #39; sights, news reports say.'
3 (Sci/Tech) -> b"'Halt science decline in schools'" b'Britain will run out of leading scientists unless science education is improved, says Professor Colin Pillinger.'
1 (Sports) -> b'Gerrard leaves practice' b'London, England (Sports Network

## Vectorizarea textului

Acum trebuie să convertim textul în **numere** care pot fi reprezentate ca tensori. Dacă dorim o reprezentare la nivel de cuvinte, trebuie să facem două lucruri:

* Folosim un **tokenizator** pentru a împărți textul în **tokeni**.
* Construim un **vocabular** din acei tokeni.

### Limitarea dimensiunii vocabularului

În exemplul dataset-ului AG News, dimensiunea vocabularului este destul de mare, mai mult de 100k cuvinte. În general, nu avem nevoie de cuvinte care apar rar în text — doar câteva propoziții le vor conține, iar modelul nu va învăța din ele. Astfel, are sens să limităm dimensiunea vocabularului la un număr mai mic, prin transmiterea unui argument constructorului vectorizatorului:

Ambele etape pot fi gestionate folosind stratul **TextVectorization**. Să instanțiem obiectul vectorizator și apoi să apelăm metoda `adapt` pentru a parcurge tot textul și a construi un vocabular:


In [5]:
vocab_size = 50000
vectorizer = keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size)
vectorizer.adapt(ds_train.take(500).map(lambda x: x['title']+' '+x['description']))

> **Notă** că folosim doar un subset al întregului set de date pentru a construi un vocabular. Facem acest lucru pentru a accelera timpul de execuție și pentru a nu te ține în așteptare. Totuși, ne asumăm riscul ca unele dintre cuvintele din întregul set de date să nu fie incluse în vocabular și să fie ignorate în timpul antrenării. Astfel, utilizarea dimensiunii întregului vocabular și parcurgerea întregului set de date în timpul `adapt` ar trebui să crească acuratețea finală, dar nu semnificativ.

Acum putem accesa vocabularul actual:


In [6]:
vocab = vectorizer.get_vocabulary()
vocab_size = len(vocab)
print(vocab[:10])
print(f"Length of vocabulary: {vocab_size}")

['', '[UNK]', 'the', 'to', 'a', 'in', 'of', 'and', 'on', 'for']
Length of vocabulary: 5335


Folosind vectorizatorul, putem codifica cu ușurință orice text într-un set de numere:


In [7]:
vectorizer('I love to play with my words')

<tf.Tensor: shape=(7,), dtype=int64, numpy=array([ 112, 3695,    3,  304,   11, 1041,    1], dtype=int64)>

## Reprezentarea textului prin metoda Bag-of-words

Deoarece cuvintele transmit semnificație, uneori putem înțelege sensul unui text doar analizând cuvintele individuale, indiferent de ordinea lor în propoziție. De exemplu, atunci când clasificăm știri, cuvinte precum *vreme* și *zăpadă* sunt susceptibile să indice *prognoza meteo*, în timp ce cuvinte precum *acțiuni* și *dolar* ar sugera *știri financiare*.

Reprezentarea vectorială **Bag-of-words** (BoW) este cea mai simplă de înțeles dintre reprezentările vectoriale tradiționale. Fiecare cuvânt este asociat unui index vectorial, iar un element al vectorului conține numărul de apariții ale fiecărui cuvânt într-un document dat.

![Imagine care arată cum este reprezentată în memorie o reprezentare vectorială bag-of-words.](../../../../../translated_images/ro/bag-of-words-example.606fc1738f1d7ba9.webp) 

> **Note**: Poți considera BoW și ca o sumă a tuturor vectorilor one-hot-encoded pentru cuvintele individuale din text.

Mai jos este un exemplu despre cum să generezi o reprezentare bag-of-words folosind biblioteca python Scikit Learn:


In [8]:
from sklearn.feature_extraction.text import CountVectorizer
sc_vectorizer = CountVectorizer()
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
sc_vectorizer.fit_transform(corpus)
sc_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[1, 1, 0, 2, 0, 0, 0, 0, 0]], dtype=int64)

Putem folosi, de asemenea, vectorizatorul Keras pe care l-am definit mai sus, transformând fiecare număr de cuvânt într-o codificare one-hot și adunând toți acei vectori:


In [9]:
def to_bow(text):
    return tf.reduce_sum(tf.one_hot(vectorizer(text),vocab_size),axis=0)

to_bow('My dog likes hot dogs on a hot day.').numpy()

array([0., 5., 0., ..., 0., 0., 0.], dtype=float32)

> **Notă**: S-ar putea să fiți surprins că rezultatul diferă de exemplul anterior. Motivul este că, în exemplul Keras, lungimea vectorului corespunde dimensiunii vocabularului, care a fost construit din întregul set de date AG News, în timp ce în exemplul Scikit Learn am construit vocabularul din textul eșantion pe loc.


## Antrenarea clasificatorului BoW

Acum că am învățat cum să construim reprezentarea bag-of-words a textului nostru, să antrenăm un clasificator care o folosește. Mai întâi, trebuie să convertim setul nostru de date într-o reprezentare bag-of-words. Acest lucru poate fi realizat folosind funcția `map` în următorul mod:


In [11]:
batch_size = 128

ds_train_bow = ds_train.map(lambda x: (to_bow(x['title']+x['description']),x['label'])).batch(batch_size)
ds_test_bow = ds_test.map(lambda x: (to_bow(x['title']+x['description']),x['label'])).batch(batch_size)

Acum să definim o rețea neuronală clasificatoare simplă care conține un singur strat liniar. Dimensiunea de intrare este `vocab_size`, iar dimensiunea de ieșire corespunde numărului de clase (4). Deoarece rezolvăm o sarcină de clasificare, funcția de activare finală este **softmax**:


In [12]:
model = keras.models.Sequential([
    keras.layers.Dense(4,activation='softmax',input_shape=(vocab_size,))
])
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train_bow,validation_data=ds_test_bow)

938/938 [==============================] - 66s 70ms/step - loss: 0.6144 - acc: 0.8427 - val_loss: 0.4416 - val_acc: 0.8697


Deoarece avem 4 clase, o acuratețe de peste 80% este un rezultat bun.

## Antrenarea unui clasificator ca o singură rețea

Pentru că vectorizatorul este, de asemenea, un strat Keras, putem defini o rețea care îl include și o putem antrena cap-coadă. În acest fel, nu mai este nevoie să vectorizăm setul de date folosind `map`, ci putem pur și simplu să trecem setul de date original la intrarea rețelei.

> **Note**: Totuși, va trebui să aplicăm mapări setului de date pentru a converti câmpurile din dicționare (cum ar fi `title`, `description` și `label`) în tupluri. Cu toate acestea, atunci când încărcăm datele de pe disc, putem construi un set de date cu structura necesară de la bun început.


In [13]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

inp = keras.Input(shape=(1,),dtype=tf.string)
x = vectorizer(inp)
x = tf.reduce_sum(tf.one_hot(x,vocab_size),axis=1)
out = keras.layers.Dense(4,activation='softmax')(x)
model = keras.models.Model(inp,out)
model.summary()

model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 tf.one_hot (TFOpLambda)     (None, None, 5335)        0         
                                                                 
 tf.math.reduce_sum (TFOpLam  (None, 5335)             0         
 bda)                                                            
                                                                 
 dense_2 (Dense)             (None, 4)                 21344     
                                                                 
Total params: 21,344
Trainable params: 21,344
Non-trainable p

## Bigrame, trigrame și n-grame

O limitare a abordării sacului de cuvinte este că unele cuvinte fac parte din expresii formate din mai multe cuvinte; de exemplu, cuvântul „hot dog” are un sens complet diferit față de cuvintele „hot” și „dog” în alte contexte. Dacă reprezentăm cuvintele „hot” și „dog” mereu folosind aceleași vectori, acest lucru poate deruta modelul nostru.

Pentru a rezolva această problemă, **reprezentările n-gram** sunt adesea utilizate în metodele de clasificare a documentelor, unde frecvența fiecărui cuvânt, pereche de cuvinte sau grup de trei cuvinte este o caracteristică utilă pentru antrenarea clasificatorilor. În reprezentările bigram, de exemplu, vom adăuga toate perechile de cuvinte în vocabular, pe lângă cuvintele originale.

Mai jos este un exemplu despre cum să generăm o reprezentare bigram a sacului de cuvinte folosind Scikit Learn:


In [14]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=1)
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
bigram_vectorizer.fit_transform(corpus)
print("Vocabulary:\n",bigram_vectorizer.vocabulary_)
bigram_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()


Vocabulary:
 {'i': 7, 'like': 11, 'hot': 4, 'dogs': 2, 'i like': 8, 'like hot': 12, 'hot dogs': 5, 'the': 16, 'dog': 0, 'ran': 14, 'fast': 3, 'the dog': 17, 'dog ran': 1, 'ran fast': 15, 'its': 9, 'outside': 13, 'its hot': 10, 'hot outside': 6}


array([[1, 0, 1, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int64)

Principalul dezavantaj al abordării n-gram este că dimensiunea vocabularului începe să crească extrem de rapid. În practică, trebuie să combinăm reprezentarea n-gram cu o tehnică de reducere a dimensionalității, cum ar fi *embeddings*, pe care o vom discuta în unitatea următoare.

Pentru a utiliza o reprezentare n-gram în setul nostru de date **AG News**, trebuie să transmitem parametrul `ngrams` constructorului nostru `TextVectorization`. Lungimea unui vocabular de bigram este **semnificativ mai mare**, în cazul nostru depășește 1,3 milioane de tokeni! Astfel, are sens să limităm și tokenii bigram la un număr rezonabil.

Am putea folosi același cod ca mai sus pentru a antrena clasificatorul, însă ar fi foarte ineficient din punct de vedere al memoriei. În unitatea următoare, vom antrena clasificatorul bigram folosind embeddings. Între timp, poți experimenta cu antrenarea clasificatorului bigram în acest notebook și să vezi dacă poți obține o acuratețe mai mare.


## Calcularea automată a vectorilor BoW

În exemplul de mai sus, am calculat vectorii BoW manual, prin însumarea codificărilor one-hot ale cuvintelor individuale. Totuși, cea mai recentă versiune a TensorFlow ne permite să calculăm vectorii BoW automat, prin transmiterea parametrului `output_mode='count` constructorului vectorizatorului. Acest lucru face definirea și antrenarea modelului nostru semnificativ mai ușoare:


In [15]:
model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_mode='count'),
    keras.layers.Dense(4,input_shape=(vocab_size,), activation='softmax')
])
print("Training vectorizer")
model.layers[0].adapt(ds_train.take(500).map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 7s 7ms/step - loss: 0.5929 - acc: 0.8486 - val_loss: 0.4168 - val_acc: 0.8772


## Frecvența termenilor - frecvența inversă a documentelor (TF-IDF)

În reprezentarea BoW, aparițiile cuvintelor sunt ponderate folosind aceeași tehnică, indiferent de cuvântul în sine. Totuși, este evident că cuvinte frecvente precum *a* și *în* sunt mult mai puțin importante pentru clasificare decât termenii specializați. În majoritatea sarcinilor NLP, unele cuvinte sunt mai relevante decât altele.

**TF-IDF** înseamnă **frecvența termenilor - frecvența inversă a documentelor**. Este o variație a modelului bag-of-words, unde, în loc de o valoare binară 0/1 care indică apariția unui cuvânt într-un document, se folosește o valoare în virgulă mobilă, care este legată de frecvența apariției cuvântului în corpus.

Mai formal, greutatea $w_{ij}$ a unui cuvânt $i$ în documentul $j$ este definită astfel:
$$
w_{ij} = tf_{ij}\times\log({N\over df_i})
$$
unde
* $tf_{ij}$ este numărul de apariții ale lui $i$ în $j$, adică valoarea BoW pe care am văzut-o anterior
* $N$ este numărul de documente din colecție
* $df_i$ este numărul de documente care conțin cuvântul $i$ în întreaga colecție

Valoarea TF-IDF $w_{ij}$ crește proporțional cu numărul de ori în care un cuvânt apare într-un document și este ajustată în funcție de numărul de documente din corpus care conțin cuvântul, ceea ce ajută la corectarea faptului că unele cuvinte apar mai frecvent decât altele. De exemplu, dacă cuvântul apare în *fiecare* document din colecție, $df_i=N$, și $w_{ij}=0$, iar acei termeni ar fi complet ignorați.

Poți crea cu ușurință o vectorizare TF-IDF a textului folosind Scikit Learn:


In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[0.43381609, 0.        , 0.43381609, 0.        , 0.65985664,
        0.43381609, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ]])

În Keras, stratul `TextVectorization` poate calcula automat frecvențele TF-IDF prin transmiterea parametrului `output_mode='tf-idf'`. Să repetăm codul pe care l-am folosit mai sus pentru a vedea dacă utilizarea TF-IDF crește acuratețea:


In [17]:
model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_mode='tf-idf'),
    keras.layers.Dense(4,input_shape=(vocab_size,), activation='softmax')
])
print("Training vectorizer")
model.layers[0].adapt(ds_train.take(500).map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 12s 12ms/step - loss: 0.4197 - acc: 0.8662 - val_loss: 0.3432 - val_acc: 0.8849


## Concluzie

Deși reprezentările TF-IDF atribuie greutăți de frecvență diferitelor cuvinte, ele nu pot reprezenta sensul sau ordinea. Așa cum a spus faimosul lingvist J. R. Firth în 1935: „Sensul complet al unui cuvânt este întotdeauna contextual, iar niciun studiu al sensului în afara contextului nu poate fi luat în serios.” Vom învăța mai târziu în curs cum să capturăm informațiile contextuale din text folosind modelarea limbajului.



---

**Declinare de responsabilitate**:  
Acest document a fost tradus folosind serviciul de traducere AI [Co-op Translator](https://github.com/Azure/co-op-translator). Deși ne străduim să asigurăm acuratețea, vă rugăm să fiți conștienți că traducerile automate pot conține erori sau inexactități. Documentul original în limba sa natală ar trebui considerat sursa autoritară. Pentru informații critice, se recomandă traducerea profesională realizată de un specialist uman. Nu ne asumăm responsabilitatea pentru eventualele neînțelegeri sau interpretări greșite care pot apărea din utilizarea acestei traduceri.
